### Multi-document summarization
#### Sanja Simonovikj, Shell internship, summer 2018

### ElasticSearch
 
ElasticSearch is a highly scalable open-source full-text search and analytics engine. It allows you to store, search, and analyze big volumes of data quickly and in near real time. It is generally used as the underlying engine/technology that powers applications that have complex search features and requirements.

The general features of Elasticsearch are as follows: 

- Elasticsearch is scalable up to petabytes of structured and unstructured data.
- Elasticsearch can be used as a replacement of document stores like MongoDB and RavenDB.
- Elasticsearch uses denormalization to improve the search performance.
- Elasticsearch is one of the popular enterprise search engines, which is currently being used by many big organizations like Wikipedia, The Guardian, StackOverflow, GitHub etc.
- Elasticsearch is open source and available under the Apache license version 2.0.

Helpful resources:
- [ElasticSearch Official Documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html) 
- [Quora answer: "How does Elasticsearch work? And how it works for data mining and big data?"](https://www.quora.com/How-does-Elasticsearch-work-And-how-it-works-for-data-mining-and-big-data)
- [Python ElasticSearch Client Official documentation ](https://elasticsearch-py.readthedocs.io/en/master/)
- [23 example queries in ElasticSearch](https://dzone.com/articles/23-useful-elasticsearch-example-queries)


NOTE: In ElasticSearch the word **document** has a special meaning. When we say "document" we do not mean a large text. Here's the definiton:

Document − It is a collection of fields in a specific manner defined in JSON format. Every document belongs to a type and resides inside an index. Every document is associated with a unique identifier, called the UID.

In our index named "sentences" we are mainly indexing sentences, along with some other fields which further help us understand the context and properties of the sentence. These fields together with the sentence field form a document in an index in ElasticSearch. The act of putting these documents from one's computer to the ElasticSearch server is called "indexing". We're currently using localhost.

Below we have an example query (the so-called "body" of the query) which we've used to generate the results, with small modifications in the numbers across different examples.

```python
# query body for the results below

body = {
  "query":{
  "bool":{

  "should":[ 
        {"match":{"sentence": string_query}},
        {"match":{"named_entities":{"query":ne_text, "boost":1.5}}},
        {"match":{"title":{"query":ne_text, "boost":0.5}}},

        ],

        "minimum_should_match": "1",
   "filter": {"range":{
         "date":{
            "gte":"2018/03/01",
            "lt":"2018/04/30"
         }
      }}
}},
"size":100,
"sort": 
            [{ "_score": { "order": "desc" }},
            {"length":{"order":"desc"}}]
}
```


What does this query do?

Let's say we're interested to know more about *"New Zealand offshore oil and gas exploration"*. This is our raw query, which we then process to get **ne_text** which is a string of the named entities in the raw query and **string_query** which is the raw query without the named entities. "sentence", "named_entities" and "title" are fields in each document indexed in elasticsearch. From the query we can see that at least two of the three match-clauses should be satisfied in order for a document to be returned as a result. Moreover, when scoring the documents, we give more importance to the "named_entities" field by using the "boost" parameter. As a simple example, the sentence *"New Zealand is a country of stunning and diverse natural beauty"* would be scored higher than *"In India, a policy to encourage enhanced oil and gas recovery projects was established to arrest declining production in the country".* However, the sentence *"Earlier, New Zealand's Prime Minister Jacinda Ardern released a statement saying that there would be "no further offshore oil and gas exploration permits granted.""* would be scored higher than both of the previous sentences.

Furthermore, we have filtered the results to include only sentences found in articles in a certain date range. We've also specified to return only the first 100 scoring sentences sorted by score and length of the sentence.

Below we're given an example indexed document. In the *_source* field we have the following sub-fields:
- **timestamp:** the moment in time when this document was put (indexed) into ElasticSearch
- **sentence:** the main field of this indexed document
- **length:** the length of the indexed sentence (given as number of words)
- **named_entities:** named entities found in the indexed sentence
- **url:** url of the news article from which the indexed sentence was extracted
- **title:** title of the news article from which the indexed sentence was extracted
- **date:** publication date of the news article from which the indexed sentence was extracted
- **prev:** the sentence found immediately before the indexed sentence in the news article
- **next:** the sentence found immediately after thes indexed sentence in the news article
- **identifier:** used to prevent indexing duplicates

``` 
{
        "_index": "sentences",
        "_type": "doc",
        "_id": "fKClh2QBi0zbGC_6dHId",
        "_score": 17.060003,
        "_source": {
          "prev": "The firm said in a recent strategic presentation that it was looking to develop Australasia as a core upstream region in the next two to five years (IOD Mar.14'18).",
          "title": "Shell Sells Remaining New Zealand Assets to OMV",
          "next": "So we want to build on that.",
          "url": "http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992739&isSearchResult=true",
          "date": "2018/03/19",
          "timestamp": "2018-07-11T10:12:04.701371",
          "length": 12,
          "sentence": "\"We have a very exciting exploration portfolio in New Zealand.",
          "named_entities": "New Zealand"
                    }
}
      
```

The goal is to use ElasticSearch to gain insights about the topic which we query. Once we query ElasticSearch we get back indexed documents like the one above, sorted by the *_score* field. From these document we can do further processing to refine the results or simply get the results in our desired format. After we get the initial results, called *hits*, we select the 6 highest scoring documents from which we extract the sentence and we also attach the previous and next sentence around the main sentence. We also provide link to the original news article as well as title and publication date. 

In order to get good results, it is crucial to have a large number of documents indexed, especially for the time period we query. It is also important to form the body of the query in a way that selects relevant documents. Finally, the raw text query should contain no more and no less than what we want to know about.

Below are some examples of how well ElasticSearch works as of now. The target information contains sentences found in an official presentation named "Deepwater Competitive Intelligence Bi-Monthly" for March/April 2018. Assuming we do not know what the exact target information is (as it's the case in the real world), we're forming a relatively general query about certain company, country or some particular aspect of their activities and we hope to get results which are related or identical to the target information.

### Example 1
Target information:
> On 12 th April 2018, New Zealand government announced that it will not offer any new offshore oil and gas
licenses, while protecting existing ones and continuing onshore licensing. 

> New Zealand to halt future offshore oil and gas drilling permits as part of its effort to combat climate  change.

> While New Zealand becomes the first country in the world to announce that it will no longer issue offshore oil and gas licenses, though protecting existing contracts and onshore licensing

We use to following query to search for information:

#### "New Zealand offshore oil and gas exploration"





Result:

Named entities, ne_text:  new zealand
Other terms, string_query:   offshore oil and gas exploration
Got 8222 Hits: 


<br/>================================================================================<br/>
  ... Our acquisition announcement involves existing production and exploration permits." Earlier, New Zealand's Prime Minister Jacinda Ardern released a statement saying that there would be "no further offshore oil and gas exploration permits granted." She said the Labour-led coalition government was "taking an important step to address climate change and create a clean, green and sustainable future for New Zealand."...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: New Zealand Springs Surprise Ban on Offshore Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995417&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... OMV said on Thursday it does not foresee any negative impact on its business from New Zealand's decision to stop issuing new permits for offshore oil and gas exploration, in a move to address climate change. Responding to the government's unexpected announcement, the Austrian major, which last month bought Royal Dutch Shell's remaining assets in the country for $578 million, told International Oil Daily: "No changes to existing permits will be made, so there is no impact at all on our purchase of Shell's assets....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: New Zealand Springs Surprise Ban on Offshore Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995417&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... The firm said in a recent strategic presentation that it was looking to develop Australasia as a core upstream region in the next two to five years (IOD Mar.14'18). "We have a very exciting exploration portfolio in New Zealand. So we want to build on that....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Shell Sells Remaining New Zealand Assets to OMV at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992739&isSearchResult=true published 2018/03/19


<br/>================================================================================<br/>
  ... But operators can still apply for an extension under a mining permit in the event of a commercial discovery, the government said. New Zealand is among the first countries to take significant steps to ban oil and gas exploration amid a growing groundswell against fossil fuel production following the Paris climate change agreement (IOD Dec.15'15). Notable campaigns include opposition to Statoil's exploration in the Great Australian Bight, protests against Total's drilling plans near the Amazon Reef and the ongoing battle against new oil sands pipelines in North America....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: New Zealand Springs Surprise Ban on Offshore Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995417&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... "Huge investments have been made by companies already anticipating offshore block offers which have now gone to waste and people's jobs will be affected," said PEPANZ head Cameron Madgwick. "It sends a worrying message to domestic and international investors on how open New Zealand is as a place to invest and create jobs." Deb Kelly, London, and Shani Alexander, Singapore...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: New Zealand Springs Surprise Ban on Offshore Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995417&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... Earlier, New Zealand's Prime Minister Jacinda Ardern released a statement saying that there would be "no further offshore oil and gas exploration permits granted." She said the Labour-led coalition government was "taking an important step to address climate change and create a clean, green and sustainable future for New Zealand." The government will limit the 2018 upstream exploration round to onshore acreage in the Taranaki Basin, Ardern said, adding it will honor all existing exploration agreements with current permit holders....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: New Zealand Springs Surprise Ban on Offshore Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995417&isSearchResult=true published 2018/04/13

### Example 2
Target information:
> Erin Energy Corp postponed exploration of Ereng and G prospects in OML 120 and OML 121 to late 2018 at earliest. The
company still required around US $ 50 mln to fund the exploration. Erin secured the “Pacific Bora” drillship in order to
perform this exploration campaign, it is now unclear if the drillship will be maintained until late year 2018.

We use to following query to search for information:

#### "Erin exploration funding"


Results:

Named entities, ne_text:  erin
Other terms, string_query:   exploration funding
Got 9490 Hits: 


<br/>================================================================================<br/>
  ... Houston-based Erin Energy and three of its divisions have filed for Chapter 11 bankruptcy protection in the US Bankruptcy Court for the Southern District of Texas, Houston Division. According to its filing, Erin Energy has between 200 and 999 creditors; total assets of 247.54 million USD; and 628.72 million USD in liabilities. The company said it is seeking a source of debtor-in-possession financing to provide it with the necessary working capital to continue its operations and move towards reorganization....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Briefly Noted at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=997009&isSearchResult=true published 2018/04/27


<br/>================================================================================<br/>
  ... The company said it is seeking a source of debtor-in-possession financing to provide it with the necessary working capital to continue its operations and move towards reorganization. Erin Energy has focused its operations in sub-Saharan Africa with an asset portfolio that consists of five licenses across three countries covering roughly 1.5 million acres, including E&P projects offshore Nigeria, as well as exploration licenses offshore Ghana and Gambia. ...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Briefly Noted at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=997009&isSearchResult=true published 2018/04/27


<br/>================================================================================<br/>
  ... The 50-50 JV is approaching all banks that have showed willingness to lend to the project, he added. The original plan involved two sources of debt funding. The first was a syndicated loan from local, regional and international commercial banks, and the second included export credit agencies (ECAs) from the UK, Italy, Spain, Japan and South Korea....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Oman Downgrade Prompts Duqm Finance Setback at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993920&isSearchResult=true published 2018/03/29


<br/>================================================================================<br/>
  ... OML 144 also has a long lease that runs until 2034. NNPC's efforts to address IOC concerns about funding issues have already borne some fruit following its 5.1 billion dollars agreement in 2016 to settle arrears on joint venture funding. That paved the way for deals with Chevron and Shell last year to invest $1.8 billion in new shallow-water schemes (EIF Jan.3'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Nigeria: Elections, Reform Limbo Delay Project Approvals at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=996527&isSearchResult=true published 2018/04/25


<br/>================================================================================<br/>
  ... Such projects generally cost less than deepwater developments, while the Sunlink block has the added attraction of being governed by a sole-risk contract. That means there is no participation by Nigerian National Petroleum Corp. (NNPC), with its perennial funding problems. It also means lower taxes than IOC-operated joint ventures with NNPC....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Nigeria: Elections, Reform Limbo Delay Project Approvals at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=996527&isSearchResult=true published 2018/04/25


<br/>================================================================================<br/>
  ... MLPs have evolved and the structure still makes sense, Alerian Index Research Director Stacey Morris said. "But things have obviously changed from the way they used to be in terms of MLPs now talking about self-funding. This old model of just issuing debt and issuing equity, building a pipeline or building a project and paying out all the cash flow is starting to change," she told EIF....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Master Limited Partnerships: Destined for Death? at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993718&isSearchResult=true published 2018/03/28


### Example 3
Target information:
> CNOOC will not have funds for any
future growth initiatives, as it is
focusing on delivery 5-6 major
projects around the world (Guyana,
Uganda, Nigeria, GOM, Canada)


We use to following query to search for information:

#### "CNOOC growth"

Results:

Named entities, ne_text:  cnooc
Other terms, string_query:   growth
Got 622 Hits: 


<br/>================================================================================<br/>
  ... CNOOC has no desire to acquire more assets because it is busy developing several promising legacy assets from its now fully integrated Canadian subsidiary Nexen. "2018 will mark the start of a multiyear growth cycle for CNOOC," analysts at Sanford Bernstein commented recently. Based on CNOOC's 2017-20 production target, analysts at Nomura calculate that the compound annual growth rate of the company's overseas production will be 7.5% while its Chinese production will decline at a compound annual rate of 1.1% during the same period....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Chinese NOCs Heed Mandate to Boost Gas Output at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=994245&isSearchResult=true published 2018/04/04


<br/>================================================================================<br/>
  ... That is a much more aggressive increase in investment than most global oil companies have announced for 2018. CNOOC has made a habit of setting conservative targets. It regularly exceeds its output goals and underspends on its capital budget....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: CNOOC Reserves Swell on Higher Prices, Overseas Projects at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=994088&isSearchResult=true published 2018/04/02


<br/>================================================================================<br/>
  ... The more than 20% increase in oil prices last year to an average above $50 per barrel had a dramatic impact on CNOOC's reserves, a critical component in investor valuations for oil companies. The impact of overseas projects also helped boost proved reserves to a record high of 4.84 billion barrels of oil equivalent for CNOOC, extending its reserve life to 10.3 years, from just 8.1 years in 2016 -- which had left many investors worried about the company's outlook. While discoveries and extensions played a role, the biggest factor in the improved reserves report was the higher oil price, which allowed CNOOC to add back 882 million boe of proven reserves it was forced to wipe off its books in 2016, when low prices rendered these assets uneconomic under reserves accounting standards....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: CNOOC Reserves Swell on Higher Prices, Overseas Projects at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=994088&isSearchResult=true published 2018/04/02


<br/>================================================================================<br/>
  ... Company sources tell World Gas Intelligence they have for now no plans to open renegotiations with sellers with the aim of reducing prices. "Even in the toughest period [before the oil price crash in 2014] we weren't able to renegotiate term prices, let alone now," a CNOOC source said. But none of the companies wants to sign more long-term LNG deals, particularly amid concerns that the start of Russian piped gas imports will reduce end-user demand for LNG....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: China LNG Importers Fret Over Price at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=996414&isSearchResult=true published 2018/04/25


<br/>================================================================================<br/>
  ... These projects are needed to increase stagnant output. CNOOC's output fell by 1.4% to 470.2 million boe (1.29 million boe/d) in 2017, driven by a 2.6% fall in its domestic production. The company foresees a return to growth, first modestly this year, to 470 million-480 million boe, and then more strongly to 500 million boe (1.37 million boe/d) by 2020....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: CNOOC Reserves Swell on Higher Prices, Overseas Projects at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=994088&isSearchResult=true published 2018/04/02


<br/>================================================================================<br/>
  ... The move is primarily aimed at improving efficiency and taking advantage of synergies within CNOOC's international portfolio. However, it may also help reduce the firm's tax bill, according to two energy equity analysts....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: CNOOC Completes Overhaul of Overseas Arm at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992431&isSearchResult=true published 2018/03/15


### Example 4
Target information:
> Vietnam PetroVietnam ordered Repsol to suspend its
“Red Emperor” project due to pressure from China,
2 nd time in less than1-year Vietnam has taken similar
action.


We use to following query to search for information:

#### "Vietnam Repsol project"

Results:

Named entities, ne_text:  vietnam
Other terms, string_query:   Repsol project
Got 38 Hits: 


<br/>================================================================================<br/>
  ... Under pressure from China, Vietnam has reportedly asked Repsol to halt drilling work at its Red Emperor upstream development -- known in Vietnamese as Ca Rong Do -- in disputed Block 07/03 in the South China Sea. This is the second time in under a year that Vietnam has ordered Repsol and its partners to stop drilling in the disputed waters. Last June, they were told to halt work in adjacent Block 136-03 after China threatened to attack Vietnamese bases in the Spratly Islands if drilling continued (IOD Jul.26'17)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Vietnam Calls a Halt to Repsol's Red Emperor at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993475&isSearchResult=true published 2018/03/26


<br/>================================================================================<br/>
  ... A Repsol subsidiary has halted a contract for a floating production, storage and offloading (FPSO) unit for the development of an offshore Vietnam block due to a force majeure event. The announcement, made by the contractors, a joint venture between Malaysian FPSO provider Yinson and Petrovietnam Technical Services Corp. (PTSC), comes just days after Vietnam reportedly asked Repsol to stop work due to pressure from China, as the block falls within disputed waters in the South China Sea (IOD Mar.26'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Repsol Calls Halt to FPSO Work in Vietnam at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993719&isSearchResult=true published 2018/03/28


<br/>================================================================================<br/>
  ... PVEP said it hoped the cooperation with Murphy Oil would help it develop small-sized oil fields in a more efficient manner amid low oil prices. Murphy Oil has already invested in other oil blocks offshore Vietnam. ...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Murphy Buys Stake in Vietnam Field at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992824&isSearchResult=true published 2018/03/19


<br/>================================================================================<br/>
  ... The project envisages a tension leg platform linked to the FPSO. Repsol has also commissioned US engineering company Keppel FloaTEC to build the tension leg platform for the project. Red Emperor is one of Repsol's key upstream projects intended to push production up to around 750,000 barrels of oil equivalent per day by 2020, from 695,000 boe/d in 2017 (IOD Mar.1'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Vietnam Calls a Halt to Repsol's Red Emperor at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993475&isSearchResult=true published 2018/03/26


<br/>================================================================================<br/>
  ... Vietnam’s politburo is reportedly discussing whether to cancel Repsol’s contract on the block outright. The unexpected demand to halt work came as Repsol was finalizing contract awards for the project following a final investment decision in the fourth quarter of 2017. Ensco reported the contract award in February for the rig, which had been idle since 2016....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Repsol Cancels Vietnam Rig Contract at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=996701&isSearchResult=true published 2018/04/25


<br/>================================================================================<br/>
  ... Under pressure from China, Vietnam has reportedly asked Repsol to halt drilling work at its Red Emperor upstream development -- known in Vietnamese as Ca Rong Do -- in disputed Block 07/03 in the South China Sea. This is the second time in under a year that Vietnam has ordered Repsol and its partners to stop drilling in the disputed waters....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Vietnam Calls a Halt to Repsol's Red Emperor at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993475&isSearchResult=true published 2018/03/26


### Example 5
Target information:
> Ghana will launch its first exploration licensing round in Q4
2018, offering six offshore blocks.

We use to following query to search for information:

#### "Ghana exploration"


Results:

Named entities, ne_text:  ghana
Other terms, string_query:   exploration
Got 7 Hits: 


<br/>================================================================================<br/>
  ... Neighboring Cote d'Ivoire and Sierra Leone are both running bid rounds this year. It would be the first formal licensing round for Ghana, which has habitually awarded exploration permits through direct negotiations. No details are currently available....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Ghana Plans First Bid Round at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995506&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... Ghana plans to hold a licensing round later this year that will offer six offshore blocks, with a second round to follow in 2019. The West African producer awarded a conditional exploration license to Exxon Mobil in January and is aiming to benefit from renewed interest in exploring the West African Transform Margin (IOD Jan.19'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Ghana Plans First Bid Round at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995506&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... Adam said they'd have to wait for the first bid round, scheduled for the fourth quarter this year. Energy Intelligence estimates Ghana produced 184,000 b/d in March from the deepwater Jubilee and Tweneboa-Enyenra-Ntomme (Ten) oil fields. Both Tano Basin projects are operated by UK independent Tullow Oil, which is partnered by Anadarko and Kosmos Energy, both of the US....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Ghana Plans First Bid Round at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995506&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... No details are currently available. Speaking to Reuters this week at the International Energy Forum in New Delhi, India, Ghana's deputy energy minister, Mohammed Amin Adam, said BP and Royal Dutch Shell were among companies expressing interest in the country. Adam said they'd have to wait for the first bid round, scheduled for the fourth quarter this year....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Ghana Plans First Bid Round at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995506&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... Ghana plans to hold a licensing round later this year that will offer six offshore blocks, with a second round to follow in 2019. The West African producer awarded a conditional exploration license to Exxon Mobil in January and is aiming to benefit from renewed interest in exploring the West African Transform Margin (IOD Jan.19'18). Neighboring Cote d'Ivoire and Sierra Leone are both running bid rounds this year....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Ghana Plans First Bid Round at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995506&isSearchResult=true published 2018/04/13


<br/>================================================================================<br/>
  ... By contrast, the big US independents are heading home. Anadarko has exited multiple West African exploration licenses; Hess is gone altogether, selling off mature production in Equatorial Guinea plus some development-ready discoveries in Ghana. European players like Tullow and Ophir are now doggedly focused on development and production projects, eschewing the exploration that made their names....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Editorial: Majors Muscle In at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995044&isSearchResult=true published 2018/04/11


#### Alternative approach

Assume we want to get a summary of important news during a certain time period, while knowing beforehand what information we're generally interested in. One way to achieve this is to combine the results by providing a list of companies or other named entities and a list of activities, such as "offshore exploration", "deep water" etc.

Let's modify the body of our query to be:

``` python
  body = {
    "query":{
    "bool":{
    "should":[ 
          {"match":{"sentence":{"query":string_query, "boost":2}}},
          {"match":{"named_entities":{"query":ne_text, "boost":5}}},
          {"match":{"title":{"query":ne_text, "boost":0.5}}},
          {"match":{"prev":{"query":ne_text, "boost":0.3}}},
          {"match":{"next":{"query":ne_text, "boost":0.3}}},
          {"match":{"prev":{"query":string_query, "boost":0.2}}},
          {"match":{"next":{"query":string_query, "boost":0.2}}}
          ],
          "minimum_should_match": "50%",
     "filter": {"range":{
           "date":{
              "gte":start_date,
              "lt":end_date
           }
        }}
  }},
  "size":100,
  "sort": 
              [{ "_score": { "order": "desc" }},
              {"length":{"order":"desc"}}]
  }
```

Furthermore, let our list of named entities be: ["Shell", "Exxon", "Chevron"] and the list of activities: ["exploration", "deep water", "drilling wells"].

The result is a combination of the highest scoring document for each query:

<br/>================================================================================<br/>
  ... The query is: Shell exploration<br/> 
Bulgaria's domestic natural gas production -- predominantly from the Galata, Kvarna and Kaliakra fields in the Black Sea -- only covers about 13% of its consumption. Bulgaria awarded Shell an exploration license for the block in February 2016 for an initial five-year term, with a two-year extension. Total expects to drill its second deepwater well in its 1-21 Khan Asparuh Block in the Bulgarian sector of the Black Sea this year....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Shell Eyes Bulgaria Exploration at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995683&isSearchResult=true published 2018/04/18
 

<br/>================================================================================<br/>
  ... The query is: Shell deep water<br/> 
After years of planning and cost-reduction efforts, Royal Dutch Shell has finally sanctioned its Vito oil development in the deepwater US Gulf of Mexico, adding to its portfolio of deepwater projects expected to be major cash generators in 2020 and beyond. Shell said Tuesday that it has taken a final investment decision (FID) to build a new production hub about 150 miles southeast of New Orleans in more than 4,000 feet of water. The Vito project -- its design simplified to make the economics work -- will have a gross peak production capacity of 100,000 barrels of oil equivalent per day....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Shell Making Long-Planned Vito Hub a Reality at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=996699&isSearchResult=true published 2018/04/25
 

<br/>================================================================================<br/>
  ... The query is: Shell drilling wells<br/> 
"Shell will invest regardless of its receivables position," says the source. Shell confirmed the plans in a statement to International Oil Daily, saying that Phase 9b will include the drilling of eight new development wells during 2018 and 2019 "as part of a wider ongoing development plan." The Egyptian government still owes Shell roughly $1 billion in arrears after years of macroeconomic pressure resulting from the 2011 revolution....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Idku Plant May Lead Egyptian LNG Export Revival at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992955&isSearchResult=true published 2018/03/20
 

<br/>================================================================================<br/>
  ... The query is: Exxon exploration<br/> 
It has a tender out to bring a third drillship to Guyana by the end of the year for a term of one to three years. The additional rig will allow Exxon to devote two rigs to drilling development wells at Liza while continuing its exploration campaign. Just last week, Exxon announced that its latest wildcat -- Pacora -- hit more than 65 feet of net pay -- boosting confidence in its ability to hit the 500,000 b/d production target....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Guyana Plays Pivotal Role in Exxon's Upstream Plans at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=991540&isSearchResult=true published 2018/03/07
 

<br/>================================================================================<br/>
  ... The query is: Exxon deep water<br/> 
The French major has agreed to acquire a 35% stake in the deepwater Canje Block from Canada's JHI Associates and local firm Mid-Atlantic Oil & Gas, which will retain a 30% interest. The tract is operated by Exxon Mobil (35%) and lies in water depths ranging from 1,700 meters-3,000 meters. Total has also bagged a 25% interest in the shallow-water Kanuku Block in water depths of 70 meters-100 meters, under an agreement with operator Repsol (37.5%), alongside partner Tullow Oil (37.5%)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Total Bags Stakes in Offshore Guyana Blocks at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=988500&isSearchResult=true published 2018/02/06
 

<br/>================================================================================<br/>
  ... The query is: Exxon drilling wells<br/> 
It has a tender out to bring a third drillship to Guyana by the end of the year for a term of one to three years. The additional rig will allow Exxon to devote two rigs to drilling development wells at Liza while continuing its exploration campaign. Just last week, Exxon announced that its latest wildcat -- Pacora -- hit more than 65 feet of net pay -- boosting confidence in its ability to hit the 500,000 b/d production target....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Guyana Plays Pivotal Role in Exxon's Upstream Plans at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=991540&isSearchResult=true published 2018/03/07
 

<br/>================================================================================<br/>
  ... The query is: Chevron exploration<br/> 
Robert Ryan, Chevron’s vice president of global exploration, said his company has slashed its exploration budget by 60% since 2014, which is in line with many of the company’s peers. Chevron has budgeted \$1.1 billion for global exploration in 2018. BHP plans to spend \$700 million this year on conventional oil exploration, down about $100 million from 2017....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Deepwater Operators Aim to Produce More Gas, Oil With Fewer Wells at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=992092&isSearchResult=true published 2018/03/12
 

<br/>================================================================================<br/>
  ... The query is: Chevron deep water<br/> 
Chevron laid out its plans to ramp up free cash flow generation in coming years as new CEO Mike Wirth stressed the supermajor’s focus on improving shareholder returns and capital discipline. Because of expected production growth, improved upstream operational efficiency and asset sales, Chevron said it will generate roughly \$14 billion of free cash flow after capital expenditures this year at \$60 per barrel Brent oil prices. That is “well in excess of our current dividend” payouts of about \$8.5 billion annually, Wirth said during the company’s annual meeting with investors....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Chevron Eyes Cash Generation, Shareholder Returns at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=991688&isSearchResult=true published 2018/03/07
 

<br/>================================================================================<br/>
  ... The query is: Chevron drilling wells<br/> 
Both wells are within tie-back distance to Chevron’s Blind Faith platform. Chevron and Total are drilling under a joint exploration agreement signed last year (IOD Sep.25'17). Chevron operates Twickenham with a 75% interest and Total owns 25%....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Chevron, Total Spud Ballymore Follow-Up in US Gulf at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995545&isSearchResult=true published 2018/04/13


## Synonyms

Sometimes we want a query such as "Exxon exploration" to give us results that do not necessarily contain the word "exploration" but they're still very related to exploration. More importantly, we want words like "Gulf of Mexico" to have the same meaning as their abbreviations "GoM".
We can incorporate synonyms into elasticsearch by configuring a "synonym" analyzer. We can construct the synonym file ourselves, and the format is as follows:
```
exxon mobil, exxonmobil, exxon
royal dutch shell, shell
usa, us, united states, united states of america
gom, gulf of mexico
deep water, deepwater

malaysia => malaysia, southeast asia, south asia, asia
brazil => brazil, south america, latin america, americas, america
nigeria => nigeria, west africa, africa
mexico => mexico, southern north america, north america, americas, america
liberia => liberia, west africa, africa
uruguay => uruguay, southeastern south america, south america, americas, america
canada => canada, northern north america, north america, americas, america
indonesia => indonesia, southeast asia, asia
sierra leone => sierra leone, west africa, africa
morocco => morocco, north africa, africa

```

Note that all characters should be lowercase, as we use lowercase analyzer before we use the synonym analyzer. Furthermore, the formatting matters and can significantly influence relevance. Read more [here](https://www.elastic.co/guide/en/elasticsearch/guide/current/synonyms-expand-or-contract.html). 
For example, above, "gom, gulf of mexico" means that the two words are completely synonymous. If we search for "GoM" we should get the same results as if we search "Gulf of Mexico". On the other hand, "malaysia => malaysia, southeast asia, south asia, asia" is called genre explansion. Instead of treating all synonyms as equal, genre expansion widens the meaning of a term to be more generic. If we search "Malasya" we'd get results solely about Malasya. If we search for "Southeast Asia" we'd get results about Southeast Asia AND Malasya. Another possibility which we haven't used yet is simple contraction, such as "United States => US". Note that if a document is indexed using this rule, any occurence of "United States" would be converted to "US". Then, if we search for "US" we'd get results for both "US" and "United States" but if we search for "United States" we wouldn't find matches as in our index the phrase "United States" has been converted to "US".

### How to use synonyms in elasticsearch?
Assume we already have an index called "sentences" that does not account for synonyms. We will create new index called "sentences25" which will use synonyms both in index and query time. Below are the steps:

#### 1) Create a txt file containing the synonym mappings, like the ones above. Put this file into the same folder as the elasticsearch configuration file ("elasticsearch.yml")

#### 2) Create the new index using the following command:

**Note:** We used Kibana console to execute the commands. This can also be done using CURL and sending requests to the elasticsearch server, the default being http://localhost:9200/. To open the Kibana console (after Kibana is installed and running) go to http://localhost:5601 --> Dev Tools --> Console.

```
PUT sentences25
{
    "settings": {
        "index" : {
            "analysis" : {
                "analyzer" : {
                    "synonym" : {
                        "tokenizer" : "whitespace",
                        "filter" : ["lowercase","graph_synonyms"]
                    }
                },
                "filter" : {
                    
                        "graph_synonyms" : {
                        "type" : "synonym_graph",
                        "synonyms_path" : "synonyms.txt"
                    }
                    
                }
            }
        }
    },
    "mappings": {
      "doc": {
        "properties": {
          "date": {
            "type": "date",
            "format": "yyyy/MM/dd"
          },
          "length": {
            "type": "long"
          },
          "named_entities": {
           "type": "text",
           "analyzer":"synonym"
	         
            
          },
          "next": {
            "type": "text",
            "analyzer":"synonym"

          },
          "prev": {
            "type": "text",
            "analyzer":"synonym"

            
          },
          "sentence": {
            "type": "text",
            "analyzer":"synonym"

            
          },
          "timestamp": {
            "type": "date"
          },
          "title": {
            "type": "text",
             "analyzer":"synonym"

          },
          "url": {
            "type": "keyword"
          },
          "identifier":{
          "type":"keyword"
          }
        }
}}}
         
```

#### 3) Trasfer the data from the "sentences" index to the "sentences25" index (this takes a few seconds to execute):
**Note:** This command does not copy the settings of the source index. It just copies the documents from the source index to the destination index. The destination index should be created and configured with the appropriate settings above before running this command.
```
POST _reindex
{
  "source": {
    "index": "sentences"
  },
  "dest": {
    "index": "sentences25"
  }
}
```

You can test whether all worked out by using the following query as an example:
```
GET sentences25/_search
{
    "query": {
        "query_string" : {
            "fields" : ["sentence","named_entities"],
            "query" : "GoM"
        }
    }
}
```
One of the results for this query is:
```
{
        "_index": "sentences25",
        "_type": "doc",
        "_id": "U6DJg2QBi0zbGC_60E5X",
        "_score": 14.229355,
        "_source": {
          "prev": "Now come the restoration, rebuilding and the continued construction of LNG export terminals that -- despite their vulnerability to tropical storm damage -- are in the right spot, analysts say.",
          "sentence": "“The siting and green-lighting of Gulf of Mexico liquefaction was no accident.",
          "named_entities": "Mexico",
          "timestamp": "2018-07-10T16:13:18.742628",
          "url": "http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=975493&isSearchResult=true",
          "length": 13,
          "next": "The GOM offers a rare convergence of proximate supply, mature infrastructure and access to seaborne offtake.",
          "title": "'Resilient' Gulf Coast LNG Industry Looks Beyond Harvey's Disruption",
          "date": "2017/10/02"
        }
      }
```
As we can see, the "sentence" field contains the word "Gulf of Mexico" even though we queried "GoM". Therefore, we have successfully accounted for synonyms in elasticsearch.


### "More Like This" - MLT Query

This is a type of query in elasticsearch that given an input text (or document id) returns similar documents. The More Like This Query finds documents that are "like" a given set of documents. In order to do so, MLT selects a set of representative terms of these input documents, forms a query using these terms, executes the query and returns the results. The user controls the input documents, how the terms should be selected and how the query is formed. Read more about how it works [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html). 

#### Example:
**For the query:** 
``` python
body = {
    "query": {
        "more_like_this" : {
            "fields" : ["sentences", "prev","next"],
            "like" : "Santos: On April 29th , Statoil spud the 1-STAT-10A-SPS new field wildcat on the pre-salt BM-S-8 Block (Santos Basin). The well is being drilled by the Seadrill’s West Saturn Drillship in 1,990m of water, and planned total depth of 6,630m. The well is also called by its pre-salt- Guanxuma prospect.",
            "min_term_freq" : 1
        }
    }
}
```
**we get:**





<br/>================================================================================<br/>
  ... The Green Canyon Block 640 field well was drilled to an unreported depth by Rowan Cos’ “Rowan Resolute” drillship. The bypass was drilled out of the company’s SS3 OCS G24179 (API 60-811-40694), which bottomed to the west beneath Block 726. It was completed as an oil producer earlier this year....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Anadarko drills another Green Canyon oil well in Caesar/Tonga field By Marc Eckhardt, Jeff Gosmano at https://penod.ihsenergy.com/ENOD/Display/1c1e76b3-ce29-4d76-850d-dcb76d559939 published 2018/07/09


<br/>================================================================================<br/>
  ... The 1 Horseshoe discovery well was drilled to a total depth of 6000 ft and encountered more than 150 ft of net oil pay in several reservoir zones in the Nanushuk section. The 1A Horseshoe sidetrack was drilled to a total depth of 8215 ft and encountered more than 100 ft of net oil pay in the Nanushuk interval as well. The Horseshoe drillsite is approximately 12 miles south of Nuiqsut, Alaska....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Several companies buy into Nanushuk prospect on North Slope By Bob Knowles at https://penod.ihsenergy.com/ENOD/Display/2168cf7c-6f86-4f59-bf0b-9eb4c6b2c04c published 2018/07/11


<br/>================================================================================<br/>
  ... Gazprom's only producing field in Bolivia is Incahuasi, launched in 2016, where production from the three-well Phase 1 totaled 2.519 Bcm last year, reaching a daily output of 8 MMcm/d, well above its 6.5 MMcm/d design capacity (IOD Aug.4'16). Phase 2 is designed to double production at Incahuasi. Total initially planned Phase 2 to be launched as early as 2017, but Gazprom was in no hurry, with CEO Alexei Miller saying in September 2016 that Phase 2 would follow in mid-2019....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Gazprom Preps for Bolivian Field Expansion at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=1001712&isSearchResult=true published 2018/06/14


<br/>================================================================================<br/>
  ... The northwestern Rawlins County venture is nearly one and a half miles west-northwest of the closest well in a contiguous field straddling the state line that’s called Scoda in Kansas and Burntwood Canyon in Nebraska. BEREXCO LLC discovered the field in southern Hitchcock County in July 2013 at the 15-31 Cheryl in se sw se 31-1n-35w. It was drilled to a total depth of 4650 ft and completed on pump for 392 bbls of oil and four bbls of oil daily from lightly acidized intervals of the Lansing-Kansas City “B” at 4130-40 ft and “E” at 4281-88 ft....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Hartman plans new exploration along Kansas/Nebraska border By Ed Marker at https://penod.ihsenergy.com/ENOD/Display/9050e010-5ff4-45cd-825d-2719f16dc499 published 2018/07/11


<br/>================================================================================<br/>
  ... The Permian Basin in West Texas and southeast New Mexico tacked on a single oil unit to push its total to 476. The Cana Woodford in Oklahoma, home of the Scoop and Stack plays, also added a single oil rig for a total of 69. The Williston Basin in North Dakota, which hosts most of the Bakken tight oil play, held steady at 57 oil rigs, while the Denver-Julesburg Basin in Colorado also held flat at 25, Baker Hughes reported....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Baker Hughes: US Oil Rig Count Flat at http://www.energyintel.com/pages/Eig_Article.aspx?DocId=1004848 published 2018/07/16


<br/>================================================================================<br/>
  ... The French major's upstream strategy is heavily reliant on offshore and deepwater projects, as well as operations in the Middle East and in global LNG. The company has a 70% interest in the operated Xerelete field, some 250 kilometers off Rio de Janiero (IOD Jul.11'12). Total also has a 20% interest in the huge ultra-deepwater Libra oil field, which it says has "considerable production potential."...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Total AGM Disrupted by Environmental Protests at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=1000550&isSearchResult=true published 2018/06/04





#### Some examples with "real" data

Let our query be:

``` python

body = {
    "query": {
        "bool":{
          "must":[
        {"query_string" : {
            "fields" : ["sentence","named_entities^1.2","prev^0.5","next^0.5","title^0.8"],
            "query" : raw
        }},
        {
        "more_like_this" : {
            "fields" : ["sentences^2", "prev","next"],
            "like" : raw,
            "min_term_freq" : 1
        }
    }
        ],
        
     "filter":{"range":{
           "date":{
              "gte":start_date,
              "lt":end_date
           }
        }}}
    },
  "size":100,
  "sort": 
              [{ "_score": { "order": "desc" }},
              {"length":{"order":"desc"}}]
}


```


### Example 6
Target information:
> BOEM Lease sale 250 generated \$US125 million in high bids, for 148 tracts covering 815,403 acres offshore GOM. A total of 33 companies submitted 159 bids. Most notable is re-investment of smaller IOC’s in the Shelf (43 bids totaling \$8Mn) due to lower royalty rate (12.5%) imposed by new Administration. The top 3 bidders, Chevron, Shell, and BP, lodged 67 of the 148 total high bids, reaffirming their commitment to the region


We use to following query to search for information:

#### "2018 GOM Lease Sale bids"

Results:


<br/>================================================================================<br/>
  ... A total of 33 companies submitted a combined 159 bids on 148 blocks, bringing in high bids totaling almost \$125 million. That was a modest improvement on the August 2017 sale in which 90 blocks received high bids totaling $121.1 million. This was the second lease sale since regulators opened up the entire Gulf for bidding in a single auction....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Shallow Water Makes Comeback in Gulf As Gas' Role Keeps Declining at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993399&isSearchResult=true published 2018/03/26


<br/>================================================================================<br/>
  ... Blocks in that area drew 43 bids in Lease Sale 250, compared to 10 in Sale 249. Celata pointed out that the royalty change was implemented shortly before Sale 249 and now operators have had more time to review prospects in those areas, but he cautioned that “one point does not make a trend.” ...
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: BP Roars Back Into US Gulf at Lease Sale at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993268&isSearchResult=true published 2018/03/22


<br/>================================================================================<br/>
  ... Meanwhile, the latest Gulf of Mexico lease sale in March was seen as disappointing by some, with total high bids amounting to \$125 million -- a fraction of the sum raised at sales in the not too distant past. However, Tommy Beaudreau of the Columbia Center on Global Energy Policy called the results "relatively healthy." BP, Chevron and Royal Dutch Shell all spent $20 million or more and picked up between a dozen and two dozen blocks a piece....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Sellers Struggle to Find Buyers for US Gulf Assets at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=995833&isSearchResult=true published 2018/04/18


<br/>================================================================================<br/>
  ... Excluded from the lease sale are: blocks subject to the congressional moratorium established by the Gulf of Mexico Energy Security Act of 2006: blocks that are adjacent to or beyond the US Exclusive Economic Zone in the area known as the northern portion of the Eastern Gap; and whole blocks and partial blocks within the current boundary of the Flower Garden Banks National Marine Sanctuary. The Gulf of Mexico OCS, covering about 160 million acres, contains an estimated 48 billion barrels of undiscovered technically recoverable oil and 141 trillion cubic feet of undiscovered technically recoverable gas. Lease Sale 250 was held in March and attracted bids on 43 shallow-water tracts, compared with 10 blocks in Lease Sale 249 held in August 2017 (NGW Mar.26'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: ROUNDUP: Mariner East 1 Still Offline as Sunoco Relocates Families Near Sinkholes at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=994340&isSearchResult=true published 2018/04/09


<br/>================================================================================<br/>
  ... There were a total of 33 companies registered to bid and they lodged a total of 159 bids. The figure is a modest improvement on Lease Sale 249 in August 2017, when companies high bids for 90 blocks totaling $121.1 million and a total of 99 bids worth \$137 million (OD Aug.17'17). However, there were more blocks available in the latest sale (OD Feb.19'18)....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: BP Roars Back Into US Gulf at Lease Sale at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993268&isSearchResult=true published 2018/03/22


<br/>================================================================================<br/>
  ... Shelf players bid on 43 shallow-water tracts in Lease Sale 250 compared to 10 blocks in Lease Sale 249 last August. Last year, the Bureau of Ocean Energy Management (BOEM) lowered the royalty rate for shallow-water blocks from 18.75% to 12.5% in a bid to spur investment in the most mature region of the Gulf. Mike Celata, BOEM’s Gulf of Mexico regional director, said operators for last week’s sale had more time to evaluate prospects in the context of the new royalty rate, which likely boosted interest....
 <br/>:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::<br/>
Extracted from: Shallow Water Makes Comeback in Gulf As Gas' Role Keeps Declining at http://www.energyintel.com/pages/SearchArticleView.aspx?DocID=993399&isSearchResult=true published 2018/03/26

